# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [3]:
data = pd.read_csv('C:/datasets/10/insurance.csv')

data.head(20)


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [4]:

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


<b>Итог изучения данных:</b> В данных отсутствуют пропуски и какие либо аномалии.

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

<b>Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)</b>

**Ответ:** не изменится

**Обоснование:** 
Вспомним, что если матрица умножается на единичную матрицу, то получается та же матрица:

Пусть 𝑃 — обратимая матрица, на которую умножаются признаки. Умножим матрицу признаков Х на обратимую матрицу Р и запишем новую формулу предсказаний.
$$
a = XPw = XP((XP)^T XP)^{-1} (XP)^T y
$$
Попробуем сократить формулу.

$$
XP((XP)^T XP)^{-1} (XP)^T y =
$$
$$
= XP(P^T X^T XP)^{-1} P^T X^T y =
$$
$$
= XPP^{-1}(X^T X)^{-1}(P^T)^{-1} P^T X^T y =
$$
$$
= XPP^{-1}(X^T X)^{-1}(P^T)^{-1}P^T X^T y =
$$

Если квардратная матрица умножается на обратную, получается единичная матрица:
$$
= XE(X^T X)^{-1}EX^T y =
$$

Вспомним, что если матрица умножается на единичную матрицу, то получается та же матрица:
$$
= X(X^T X)^{-1}X^T y 
$$

Полученная в результате сокращения формула равна изначальной формуле предсказания:
$$
a = Xw = Х(X^T X)^{-1} X^T y
$$

## Алгоритм преобразования

**Алгоритм**

В качестве алгоритма преобразования данных для решения задачи предлагаю умножение на обратимую матрицу.

**Обоснование**

При умножениии на обратимую матрицу качество линейной регрессии не поменяется, что доказано выше, во втором пункте "Умножение матриц".

## Проверка алгоритма

In [5]:
# Создаю обучающую и тестовую выборки

target = data['Страховые выплаты']
features = data.drop('Страховые выплаты', axis=1)

In [14]:
# Создаю функцию для шифрования признаков

def encrypt(features):
    inversable_matrix = np.random.normal(0, 1, (features.shape[1], features.shape[1]))
    features_encrypted = features @ inversable_matrix
    return features_encrypted

In [21]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.4249455028666801


In [20]:
features_encrypted = encrypt(features)

model_encrypted = LinearRegression()
model_encrypted.fit(features_encrypted, target)
predictions_encrypted = model_encrypted.predict(features_encrypted)
print(r2_score(target, predictions_encrypted))

0.4249455028666833


<b>Вывод:</b>

Таким образом, после преобразования умножением признаков на обратимую матрицу качество линейной регрессии из sklearn не отличается до и после преобразования, что подтверждается метрикой R2.

## Итоги проекта


<b>Итог изучения данных:</b> В данных отсутствуют пропуски и какие либо аномалии.

Было установлено, что при умножении признаков на обратимую матрицу качество линейной регрессии не изменится, поэтому в качестве алгоритма преобразования данных для решения задачи можно использовать умножение на обратимую матрицу.

Таким образом, после преобразования умножением признаков на обратимую матрицу качество линейной регрессии из sklearn не отличается до и после преобразования, что подтверждается метрикой R2:

- R2 линейной регрессии до шифрования: <b>0.4249455028666801</b>
- R2 линейной регрессии после шифрования: <b>0.4249455028666833</b>
